<div style="background-color: black; color: white; padding: 10px;text-align: center;">
  <strong>Date Published:</strong> October 23, 2025 <strong>Author:</strong> Adnan Alaref
</div>

# **🚀 AutoGrad Engine**
A lightweight, educational, and fully functional automatic differentiation engine built from scratch using pure Python — inspired by the design philosophy of micrograd by **karpathy** , but extended with Torch-like features, dynamic learning rate, and graph visualization.

This implementation introduces **PyTorch-inspired design patterns, including a Module hierarchy, dynamic forward graph construction, integrated activation functions, and adaptive learning rate logic**. It bridges the **gap between simplicity and modern deep learning framework structure** — giving learners both transparency and realistic functionality.



# 🧠 Overview
This Notebook implements a minimal deep learning core framework that includes:


- **Custom scalar-based automatic differentiation engine.**  
- **Neural network building blocks (Neuron, Layer, MLP).**
- **Dynamic learning rate scheduling during training.**
- **Graphviz-based visualization of the computational graph.**

> The code is intentionally simple, readable, and modular — making it perfect for understanding the internal mechanics of backpropagation and gradient descent.

#### 🔗You can go to view all project in github [**Scaler Autograd Engine**](https://github.com/Adnan-Alaref/Autograd-engine-scalar-automatic-differentiation-engine.git).
---

# **⚙️ Core Components**

## 1. Value (`engine.py`)
The `Value` class represents a single scalar value in the computational graph.  
It supports arithmetic operations, activation functions, and automatic differentiation through a dynamically constructed graph.

### Key Features
- **Operator Overloading:** Supports `+`, `-`, `*`, `/`, `**`, negation.  
- **Nonlinearities:** `tanh()`, `relu()`, `sigmoid()`, `exp()`, `log()`, `abs()`.  
- **Backpropagation:** Uses a DFS-based topological traversal for gradient computation.  
- **Autocasting:** Accepts Python floats or 0D torch tensors.  
- **Stability Enhancements:** Safe handling for division, log, and exponentiation edge cases.  

### Example
```python
a = Value(2.0)
b = Value(3.0)
c = a * b + a.tanh()
c.backward()

print(c.data)      # Forward result
print(a.grad, b.grad)  # Computed gradients
```
---

## 2. Module, Neuron, Layer, and MLP (`nn.py`)
These classes provide a minimal neural network framework built entirely on top of the `Value` engine.

### Module
Base class for all trainable models.  

**Includes:**
- Learning rate and patience tracking  
- Dynamic LR adjustment based on performance  
- Unified `train()` / `eval()` modes (similar to PyTorch)  
- Zeroing gradients via `zero_grad()`  

### Neuron
Represents a single fully connected neuron:

```python
n = Neuron(nin=3, act_func="relu")
```
- Each neuron maintains its own weights, bias, and activation function.

### Layer
Wraps multiple neurons into a dense layer:

```python
layer = Layer(nin=3, nout=4, act_func="tanh")
inputs = [Value(1), Value(2), Value(3)]
out = layer(inputs)
```

### MLP (Multi-Layer Perceptron)
Stacks layers sequentially to form a deep neural network:

```python
model = MLP(nin=3, nouts=[4, 4, 1], act_func="relu")
```
### Additional Features
- `summary()` prints a readable model overview  
- Adaptive learning rate logic inside `update(loss)`  

---

## 3. Graph Visualization (`build_graph.py`)
Uses **Graphviz** to render the computational graph created during forward and backward passes.

---

# **Step 1: Import Library.**

In [1]:
import math
import torch
import random
import numbers
import numpy as np
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

# **Step 2: Create Engine.**

In [2]:
class Value:
  '''shared across all instances use in train/eval mode'''
  grad_enabled = True

  '''save memory and speed up attribute access than __dict__.'''
  __slots__ = ('data', '_prev', '_op', 'grad', '_backward', 'label')

  def __init__(self, data, _children=() , _op='', label=''):

    # --- Auto-cast logic ---
    if isinstance(data, torch.Tensor):
      data = data.item() # extract float from 0-dim tensor
    elif isinstance(data,(list,tuple)):
       raise TypeError("Value class supports only scalar data, not lists or arrays.")
    elif not isinstance(data, numbers.Real):
      raise TypeError(f"Unsupported data type {type(data)} for Value.")

    self.data = float(data)
    self.grad = 0.0
    self._op = _op
    self.label = label
    self._prev = set(_children)
    self._backward = lambda:None

  # --- Basic arithmetic operations ---
  '''Support Basic arithmetic operations either Value or a number'''

  # Addition
  def __add__(self, other):
    is_other_value = isinstance(other, Value)
    if is_other_value:
      out = Value(self.data + other.data, (self,other), '+')
    else:
      out = Value(self.data + other, (self,), '+')

    if Value.grad_enabled:
      def _backward():
        self.grad += 1.0 * out.grad
        if is_other_value:
          other.grad += 1.0 * out.grad
      out._backward = _backward
    return out

  def __radd__(self, other):
    # 1 + x → int.__add__(1, x) → (fails) → x.__radd__(1)
    return self.__add__(other)

  # Subtraction
  def __sub__(self, other):
    is_other_value = isinstance(other, Value)
    if is_other_value:
      out = Value(self.data - other.data, (self, other), '-')
    else:
      out = Value(self.data - other, (self,), '-')

    if Value.grad_enabled:
      def _backward():
          self.grad += 1.0 * out.grad
          if is_other_value:
            other.grad += -1.0 * out.grad
      out._backward = _backward
    return out

  def __rsub__(self, other):
    # 1 - x → int.__sub__(1,x) → (fails) → x.__rsub__(1)
    is_other_value = isinstance(other, Value)
    if is_other_value:
      out = Value(other.data - self.data, (other, self), '-')
    else:
      out = Value(other - self.data, (self,), '-')

    if Value.grad_enabled:
      def _backward():
        self.grad += -1.0 * out.grad
        if is_other_value:
          other.grad += 1.0 * out.grad
      out._backward = _backward
    return out

  # Multiplication
  def __mul__(self, other):
    is_other_value = isinstance(other, Value)
    if is_other_value:
      out = Value(self.data * other.data, (self,other), '*')
    else:
      out = Value(self.data * other, (self,), '*')

    if Value.grad_enabled:
      def _backward():
        self.grad += (other.data if is_other_value else other) * out.grad
        if is_other_value:
          other.grad += self.data * out.grad
      out._backward = _backward
    return out

  def __rmul__(self, other):
    # 1*x → int.__mul__(1,x) → (fails) → x.__rmul__(1)
    return self.__mul__(other)

  # True Division
  def __truediv__(self, other):
    # a/b = a*b**-1
    is_other_value = isinstance(other, Value)
    if is_other_value:
      if other.data == 0:
          raise ZeroDivisionError("division by zero")
      out = Value(self.data * (other.data**-1), (self, other), '/')

    else:
      if other == 0:
            raise ZeroDivisionError("division by zero")
      out = Value(self.data * (other**-1), (self,), '/')

    if Value.grad_enabled:
      def _backward():
        if is_other_value:
          # numerical regularization or epsilon stabilization
          safe_other = other.data if abs(other.data) > 1e-12 else 1e-12

          self.grad += (1 / safe_other) * out.grad
          other.grad += ((-self.data)/(safe_other**2)) * out.grad

        else:
          # a / c = d(a/c)/da = 1/c
          self.grad +=(1/other) * out.grad
      out._backward = _backward
    return out

  def __rtruediv__(self,other):
    # 1 / x → int.__truediv__(1, x) → (fails) → x.__rtruediv__(1)
    return Value(other) * (self**-1)

  # Power
  def __pow__(self, other):
    is_other_value = isinstance(other, Value)
    if is_other_value:
      out = Value(self.data** other.data, (self,other), f'**{other}')

    else:
      out = Value(self.data** other, (self,), f'**{other}')

    if Value.grad_enabled:
      def _backward():
        if is_other_value:
          # 1. Update gradient for the base (self) ∂f/∂x = y * x^(y-1)
          self.grad += other.data * (self.data**(other.data-1)) * out.grad

          # 2. Update gradient for the exponent (other) ∂f/∂y = x^y * ln(x)
          '''Use safe base to avoid log(0) or negative issues'''
          base = self.data if self.data > 0 else 1e-8
          other.grad += (self.data**other.data) * math.log(base) * out.grad

        else:
          # When other is constant: d(x^a)/dx = a * x^(a-1)
          self.grad += other * (self.data**(other-1)) * out.grad
      out._backward = _backward
    return out

  def __rpow__(self, other):
    # 1**x → int.__pow__(1,x) → (fails) → x.__rpow__(1)
    base = other
    out = Value(base**self.data, (self,), f'**{base}' )

    if Value.grad_enabled:
      def _backward():
        safe_base = base if base > 0 else 1e-8
        self.grad += (base**self.data) * math.log(safe_base) * out.grad
      out._backward = _backward
    return out

  # Ngative numbers
  def __neg__(self):
    out = Value(self.data *-1, (self,) ,'neg')

    if Value.grad_enabled:
      def _backward():
        self.grad += -1 * out.grad
      out._backward = _backward
    return out

  # --- Activation functions (non linearity)---

  # Tanh
  def tanh(self):
    x = self.data
    t_val = math.tanh(x)
    # t_val = (math.exp(2*x) -1)/(math.exp(2*x) +1)
    out = Value(t_val, (self,), 'Tanh')

    if Value.grad_enabled:
      def _backward():
        self.grad += (1 - t_val**2) * out.grad
      out._backward = _backward
    return out

  # Rule
  def relu(self):
    r_val = self.data if self.data > 0 else 0.0
    out = Value(r_val, (self,), 'RelU')

    if Value.grad_enabled:
      def _backward():
        self.grad += (1.0 if self.data > 0 else 0.0)  * out.grad
      out._backward = _backward
    return out

  # Sigmoid
  def sigmoid(self):
    s_val = 1 / (1+math.exp(-self.data))
    out = Value(s_val, (self,), 'Sigmoid')

    if Value.grad_enabled:
      def _backward():
        self.grad += s_val * (1-s_val) * out.grad
      out._backward = _backward
    return out

  # Exponential
  def exp(self):
    e_val = math.exp(min(self.data, 50))
    out = Value(e_val, (self,), 'exp')

    if Value.grad_enabled:
      def _backward():
        self.grad += e_val * out.grad
      out._backward = _backward
    return out

  # Log
  def log(self):
    if self.data <= 0:
      raise ValueError("Cannot calculate the natural log (ln) for non-positive numbers (x <= 0).")

    log_val = math.log(self.data)
    out = Value(log_val, (self,), 'log')

    if Value.grad_enabled:
      def _backward():
        self.grad += (1 / self.data) * out.grad
      out._backward = _backward
    return out

  # Absolute
  def abs(self):
    abs_val = self.data if self.data >= 0 else -self.data
    out = Value(abs_val, (self,), 'abs')

    if Value.grad_enabled:
      def _backward():
        self.grad +=(1 if self.data>0 else -1 if self.data < 0 else 0) *out.grad
      out._backward = _backward
    return out
  
  # Backpropagation
  def backward(self):
    stack = []
    visted = set()

    def dfs(root):
      if root in visted:
        return
      visted.add(root)
      for _child in root._prev:
        dfs(_child)
      stack.append(root)

    # Build topological order
    dfs(self)

    # Backward pass
    self.grad = 1.0
    for node in reversed(stack):
      node._backward()

  def to_tensor(self):
    return torch.tensor(self.data, dtype=torch.float32, requires_grad=False)

  def __repr__(self) -> str:
    return f"Value(data={self.data})"

# **Step 3: Create nn(api top of engine).**

In [3]:
class Module:
  __slots__ = ('lr', 'best_loss', 'training', 'patience')
  def __init__(self):
    self.lr = 0.1
    self.patience = 0
    self.training = True
    self.best_loss = float('inf')

  def parameters(self):
    return []

  def children(self):
    return []

  def train(self):
    self.training = True
    Value.grad_enabled = True
    for _child in self.children():
      _child.train()

  def eval(self):
    self.training = False
    Value.grad_enabled = False
    for _child in self.children():
      _child.eval()

  def zero_grad(self):
    for p in self.parameters():
      p.grad = 0.0 if hasattr(p,'grad') else None

  def update(self, loss:float):
    # --- Update parameters ---
    for p in self.parameters():
      p.data -= self.lr * p.grad

    # --- Dynamic LR adjustment ---
    if loss.data < self.best_loss:
      self.best_loss = loss.data
      self.patience = 0
      self.lr *=1.02 # small increase if improving

    else:
      self.patience +=1
      if self.patience >= 3: # wait 3 bad steps before reducing
        self.lr *=0.8
        self.patience = 0

    # clamp LR to avoid explosion
    self.lr = max(min(self.lr,1.0),1e-4)

class Neuron(Module):
  __slots__ = ('weights','bias', "act_func")

  def __init__(self,nin:int, act_func="relu"):
    super().__init__()
    self.act_func = act_func
    self.bias = Value(random.uniform(-1, 1), label='b')
    self.weights = [Value(random.uniform(-1, 1),label=f'w{k}') for k in range(nin)]

  def __call__(self, x):
    return self.activate(x)

  def forward(self,x:list['Value'])->Value:
    # wi * xi + b
    net = sum((wi*xi for (wi,xi)in zip(self.weights, x)), start=self.bias)
    return net

  def activate(self, x):
    out = self.forward(x)
    if self.act_func==None:
      return out # row output

    activations = {
        "tanh":out.tanh,
        "relu":out.relu,
        "sigmoid":out.sigmoid
    }
    if self.act_func not in activations:
      raise ValueError(f"Unknown activation function: {self.act_func}")
    return activations[self.act_func]()

  def parameters(self):
    """Return all trainable parameters in neuron"""
    return self.weights + [self.bias]

class Layer(Module):
  __slots__ = ("nin", "nout", "neurons", "act_func")

  def __init__(self, nin, nout, act_func="relu"):
    super().__init__()
    self.nin = nin
    self.nout = nout
    self.act_func = act_func
    self.neurons = [Neuron(self.nin, act_func=self.act_func) for _ in range(self.nout)]

  def __call__(self,x):
    return self.forward(x)

  def children(self):
    return self.neurons

  def forward(self, x:list['Value'])->Value:
    nets = [neuron(x) for neuron in self.neurons]
    return nets[0] if len(nets)==1 else nets

  def parameters(self):
    """Return all trainable parameters from all neurons in the layer"""
    return list(chain.from_iterable(n.parameters() for n in self.neurons))

class MLP(Module):
  __slots__ = ("nin", "nouts", "sz", "layers","act_func")

  def __init__(self, nin:int, nouts:list, act_func="relu"):
    super().__init__()
    self.sz = [nin] + nouts
    self.act_func = act_func
    self.layers = [Layer(nin=self.sz[i], nout=self.sz[i+1],
                      act_func=(self.act_func if i<len(nouts)-1 else None)) for i in range(len(nouts))]

  def __call__(self,x):
    return self.forward(x)

  def children(self):
    return self.layers

  def forward(self,x:list['Value'])->Value:
    for layer in self.layers:
      x = layer(x) # feed output of one layer to the next
    return x

  def parameters(self):
    return list(chain.from_iterable(layer.parameters() for layer in self.layers))

  def summary(self):
    print("=================================================")
    print(f"{'Layer (type)':20} {'Output Shape':20} {'Param #':10}")
    print("=================================================")
    total_params = 0

    for i, layer in enumerate(self.layers):
        layer_name = layer.__class__.__name__
        out_shape = f"[{layer.nout}]"
        param_count = len(layer.parameters())
        total_params += param_count
        print(f"Layer-{i+1} ({layer.nin:2d} → {layer.nout:2d})".ljust(25) +\
              f"{out_shape:<20}{param_count:<10}")

    print("=================================================")
    print(f"Total params: {total_params}")
    print("=================================================")

# **Step 4: Build Computional Graph.**

In [4]:
from graphviz import Digraph

def trace(root):

  # builds a set of all nodes and edges (set(tuple)) in a graph
  nodes, edges = set(), set()
  
  def build_graph(n):
    if n not in nodes:
      nodes.add(n)
      for _child in n._prev:
        edges.add((_child,n))
        build_graph(_child)
  build_graph(root)
  return nodes, edges

def draw_graph(root):
  dot = Digraph(name="AutoGrad",
                format='svg',
                comment="Computational Graph",
                graph_attr={'rankdir':'LR', 'bgcolor': 'white'},
                edge_attr={'penwidth': '1', 'color': 'black', 'arrowhead': 'vee'})

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n)) # Unique memmory address(Identifier)
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name=uid, label="{%s | data %.4f | grad %.4f}"%(n.label, n.data, n.grad), shape='record', fillcolor='lightgray')
    # dot.node(name=uid, label=f"{n.label} | {n.data:.4f} | {n.grad:.4f}", shape='Mrecord',fillcolor='lightgray')

    if n._op:
       # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op ,label = n._op)
       # and connect this node to it
      dot.edge(tail_name=uid+n._op, head_name=uid)

  for (e1, e2) in edges:
    # connect n1 to the op node of n2
    head = str(id(e2)) + (e2._op if e2._op else '')
    dot.edge(tail_name=str(id(e1)), head_name=head)
  
  # Generate multiple output formats
  dot.render('graph_output', format='png', cleanup=True)
  return dot

# **Step 5: Test Engine.**  
## **✅ Test 1: test_sanity_check()**
- Purpose: Basic sanity test — checks whether your autograd engine correctly computes forward values and backward gradients for a simple expression.

In [5]:
def test_sanity_check():

    x = Value(-4.0)
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    assert xmg.grad == xpt.grad.item()

# Call
test_sanity_check()

## 🧮 Test 2: test_more_ops()
- Purpose: Complex expression test — ensures your implementation handles mixed operations, chaining, broadcasting, and nonlinearities (ReLU).
- This test runs a longer and more complicated sequence of operations, involving:
>Addition, multiplication, power (**), ReLU, and division.

In [6]:
def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a).relu()
    d += 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol

# Call
test_more_ops()

In [7]:
a = Value(-4.0)
b = Value(2.0)
c = a + b
d = a * b + b**3
c += c + 1
c += 1 + c + (-a)
d += d * 2 + (b + a).relu()
d += 3 * d + (b - a).relu()
e = c - d
f = e**2
g = f / 2.0
g += 10.0 / f
print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass
g.backward()
print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db

24.7041
138.8338
645.5773


# **Step 6: Simple demo.**
## **Preper Data.**

In [8]:
X = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

# Test data (unseen samples)
X_test = [
    [2.0, 2.0, -0.5],
    [1.5, -0.5, 1.0],
    [0.0, 1.0, 0.5],
    [3.0, 0.0, -1.0],
]
ys_test = [1.0, -1.0, -1.0, 1.0] # desired targets
print(f'X size: {len(X) ,len(X[0])}')

X size: (4, 3)


## **Build Model.**

In [9]:
model = MLP(nin=len(X[0]),nouts=[10,10,1])

In [10]:
model.summary()

Layer (type)         Output Shape         Param #   
Layer-1 ( 3 → 10)        [10]                40        
Layer-2 (10 → 10)        [10]                110       
Layer-3 (10 →  1)        [1]                 11        
Total params: 161


## **Gnerate Computional graph.**

In [11]:
ypreds = [model(x) for x in X]
loss = sum((yt - yp)**2 for yt,yp in zip(ys,ypreds))
loss.backward()
loss.label='L'
draw_graph(loss)

<a id="Import"></a>
<p style="background-color: #000000; font-family: 'Verdana', sans-serif; color: #FFFFFF; font-size: 160%; text-align: center; border-radius: 25px; padding: 12px 20px; margin-top: 20px; border: 2px solid transparent; background-image: linear-gradient(black, black), linear-gradient(45deg, #FF00FF, #00FFFF, #FFFF00, #FF4500); background-origin: border-box; background-clip: content-box, border-box; box-shadow: 0px 4px 20px rgba(255, 105, 180, 0.8);">
   Thanks & Upvote ❤️</p>